Import Libraries

In [147]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine


 Extract Data Function


In [149]:
def extract_data(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/114.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9",
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        print("Successfully connected to the website!")
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

Parse Data Function

In [155]:
def parse_data(soup):
    if not soup:
        print("No data to parse.")
        return []

    products = []
    items = soup.find_all('li', class_='wt-list-unstyled') 
    for item in items:
        try:
            name = item.find('h3', class_='wt-text-caption').text.strip()
            price = item.find('span', class_='currency-value').text.strip()
            description = item.find('p', class_='wt-text-caption').text.strip() if item.find('p') else ""
            products.append({"Name": name, "Price": price, "Description": description})
        except AttributeError:
            continue

    print(f"Extracted {len(products)} products.")
    return products

Clean the Data

In [156]:
def clean_data(raw_data):
    if not raw_data:
        print("No data to clean.")
        return pd.DataFrame()

    df = pd.DataFrame(raw_data)
    try:
        df['Price'] = df['Price'].str.replace(',', '').astype(float, errors='ignore')
        df.drop_duplicates(inplace=True)
        df.dropna(inplace=True)
    except Exception as e:
        print(f"Error cleaning data: {e}")
    return df